# PyTorch Model

In [1]:
import torch
import torch.nn as nn
import numpy as np
import math

from settings import *
from utils import load_np_arr, save_tensor

In [2]:
dev = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {dev} device")

Using cuda device


## Data preparation

Load and convert the data into tensors

In [3]:
Y = load_np_arr(Y_FILE_NAME)
R = load_np_arr(R_FILE_NAME)
mu = load_np_arr(MU_FILE_NAME)

Data info

In [4]:
num_features = 100
num_movies = R.shape[0]
num_users = R.shape[1]

print('Num. features:', num_features)
print('Num. movies:  ', num_movies)
print('Num. users:   ', num_users)

Num. features: 100
Num. movies:   9724
Num. users:    610


Manually rate some movies by their id

In [5]:
my_ratings = np.zeros(num_movies)

my_ratings[314] = 5 # Forrest Gump
my_ratings[461] = 5 # Schindler's List

my_ratings[904] = 5 # 12 Angry Men
my_ratings[7315] = 5 # 12 Angry Men too! we rate them both just in case

my_ratings[3635] = 5 # A Beautiful Mind (2001)
my_ratings[8240] = 5 # Rush (2013)
my_ratings[8448] = 5 # Whiplash (2014)
my_ratings[9363] = 4 # Your Name. (2016)

my_ratings[7675] = 0.5 # The Avengers (2012)
my_ratings[8668] = 0.5 # Avengers: Age of Ultron
my_ratings[8675] = 0.5 # Avengers: Infinity War - Part I
my_ratings[1985] = 0.5 # Superman
my_ratings[1986] = 0.5 # Superman II
my_ratings[1987] = 0.5 # Superman III
my_ratings[1988] = 0.5 # Superman IV: The Quest for Peace

Append `my_ratings` to the `R` amd `Y`

In [6]:
# Append the normalized my_ratings to Y
Y = np.c_[Y, my_ratings - mu.squeeze()]

# Append the binary my_ratings to R
R = np.c_[R, (my_ratings != 0).astype(int)]

Increment `num_users` after appending new user

In [7]:
num_users += 1

## Collaborative filtering cost function

|General <br />  Notation  | Description                                                                | Python (if any) |
|:-------------------------|:---------------------------------------------------------------------------|-----------------|
| $r(i,j)$                 | scalar; = 1  if user j rated movie i  = 0  otherwise                       |                 |
| $y(i,j)$                 | scalar; = rating given by user j on movie  i    (if r(i,j) = 1 is defined) |                 |
| $\mathbf{w}^{(j)}$       | vector; parameters for user j                                              |                 |
| $b^{(j)}$                | scalar; parameter for user j                                               |                 |
| $\mathbf{x}^{(i)}$       | vector; feature ratings for movie i                                        |                 |     
| $n_m$                    | number of movies                                                           | num_movies      |
| $n_u$                    | number of users                                                            | num_users       |
| $n$                      | number of features                                                         | num_features    |
| $\mathbf{X}$             | matrix of vectors $\mathbf{x}^{(i)}$                                       | X               |
| $\mathbf{W}$             | matrix of vectors $\mathbf{w}^{(j)}$                                       | W               |
| $\mathbf{b}$             | vector of bias parameters $b^{(j)}$                                        | b               |
| $\mathbf{R}$             | matrix of elements $r(i,j)$                                                | R               |

The collaborative filtering cost function is given by

$$
J({\mathbf{x}^{(0)},\dots,\mathbf{x}^{(n_m-1)},\mathbf{w}^{(0)},b^{(0)},\dots,\mathbf{w}^{(n_u-1)},b^{(n_u-1)}}) = \left[\frac{1}{2}\sum_{(i,j):r(i,j)=1} (\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right] + \underbrace{\left[\frac{\lambda}{2} \sum^{n_u-1}_{j=0} \sum^{n-1}_{k=0} (\mathbf{w}^{(j)}_k)^2 + \frac{\lambda}{2} \sum^{n_m-1}_{i=0} \sum^{n-1}_{k=0} (\mathbf{x}^{(i)}_k)^2 \right]}_{\text{regularization}}
$$

The first summation in (1) is "for all $i$, $j$ where $r(i,j)$ equals $1$" and could be written:

$$
= \left[ \frac{1}{2}\sum_{j=0}^{n_u-1} \sum_{i=0}^{n_m-1}r(i,j)*(\mathbf{w}^{(j)} \cdot \mathbf{x}^{(i)} + b^{(j)} - y^{(i,j)})^2 \right] + \text{regularization}
$$

In [8]:
def cofi_cost_func(X: torch.Tensor, W: torch.Tensor, b: torch.Tensor,
                   Y: torch.Tensor, R: torch.Tensor, lambda_: torch.float) -> torch.float:
    z = torch.matmul(X, W.T)
    z = z.add(b)
    j = z.subtract(Y)
    j = j.pow(2)
    j = j.multiply(R)
    j = j.sum()
    j = j.multiply(0.5)

    reg_val = (lambda_ / 2) * (W.pow(2).sum() + X.pow(2).sum())

    return j + reg_val

## Model

Convert `R` and `Y` to tensors

In [9]:
Y = torch.tensor(Y, device=dev)
R = torch.tensor(R, device=dev)

Create and initialize parameters

In [10]:
X = nn.Parameter(
    torch.Tensor(num_movies, num_features).to(dev)
)
W = nn.Parameter(
    torch.Tensor(num_users, num_features).to(dev)
)
b = nn.Parameter(
    torch.Tensor(1, num_users).to(dev)
)


# Initialize parameters
nn.init.kaiming_uniform_(X, a=math.sqrt(5)).to(dev)
nn.init.kaiming_uniform_(W, a=math.sqrt(5)).to(dev)
fan_in, _ = nn.init._calculate_fan_in_and_fan_out(W)
bound = 1 / math.sqrt(fan_in)
nn.init.uniform_(b, -bound, bound).to(dev)

print(f'Initial loss: {cofi_cost_func(X, W, b, Y, R, lambda_=1)}')

Initial loss: 40497.382090237494


Create optimizer

In [11]:
optimizer = torch.optim.Adam(params=[X, W, b], lr=1e-4)

Training loop

In [12]:
epochs = 15001
lambda_ = 1

print('Epoch       Loss')

for epoch in range(1, epochs):
    cost_val = cofi_cost_func(X, W, b, Y, R, lambda_)

    optimizer.zero_grad()
    cost_val.backward()

    optimizer.step()

    if epoch % 100 == 0:
        print(f'{epoch:5}       {cost_val:.5f}')

Epoch       Loss


  100       38971.61455
  200       37316.92164
  300       35337.73749
  400       33002.74322
  500       30440.43454
  600       27849.04441
  700       25375.66029
  800       23083.31637
  900       20988.91764
 1000       19094.52744
 1100       17396.71030
 1200       15887.60845
 1300       14555.35046
 1400       13385.12150
 1500       12360.61044
 1600       11465.30948
 1700       10683.42040
 1800       10000.37468
 1900       9403.07429
 2000       8879.94450
 2100       8420.88098
 2200       8017.13122
 2300       7661.15175
 2400       7346.46179
 2500       7067.49636
 2600       6819.48263
 2700       6598.32522
 2800       6400.51126
 2900       6223.02951
 3000       6063.30095
 3100       5919.11495
 3200       5788.57869
 3300       5670.06741
 3400       5562.18238
 3500       5463.71822
 3600       5373.63161
 3700       5291.01414
 3800       5215.07277
 3900       5145.11272
 4000       5080.52288
 4100       5020.76222
 4200       4965.35335
 4300       4913

Save the parameters

In [13]:
print(save_tensor(W, W_FILE_NAME))
print(save_tensor(X, X_FILE_NAME))
print(save_tensor(b, B_FILE_NAME))

Saved tensor in ./cache/W.pt
Saved tensor in ./cache/X.pt
Saved tensor in ./cache/b.pt


## Inference

...